# Blue Brain BioExplorer

![](./images/bioexplorer_banner.png)

### Prerequiries

In [ ]:
!pip install pyquaternion

### Initialization

In [ ]:
from bioexplorer import BioExplorer, Protein, Surfactant, Membrane, Volume, \
                        Cell, Sugars, Vector2, Vector3, Quaternion
from pyquaternion import Quaternion as pyQuaterion
from braynsmediamaker import MovieMaker
import nglview

be = BioExplorer('localhost:5000')
core = be.core_api()
mm = MovieMaker(core)
status = be.reset()

### Resources

In [ ]:
resource_folder = '../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'

glucose_path = pdb_folder + 'glucose.pdb'
lactoferrin_path=pdb_folder + 'immune/1b0l.pdb'
defensin_path = pdb_folder + 'immune/1ijv.pdb'

surfactant_head_source = pdb_folder + 'surfactant/1pw9.pdb'
surfactant_branch_source = pdb_folder + 'surfactant/1k6f.pdb'

### Configuration

In [ ]:
# Scene
scene_size = 250

# Proteins
protein_radius_multiplier = 1.0
protein_representation = be.REPRESENTATION_ATOMS
protein_load_hydrogen = False

# Immune system
nb_glucoses = 3600
nb_lactoferrins = 10
nb_defensins = 30

# Virus
add_glycans = True

## Camera and rendering settings

In [ ]:
status = be.core_api().set_camera(
    orientation=[0.0, 0.0, 0.0, 1.0],
    position=[4.883, 44.255, 431.911],
    target=[4.883, 44.255, 31.311]
)

In [ ]:
be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer',
    samples_per_pixel=1, subsampling=4, max_accum_frames=64)
params = be.core_api().BioExplorerRendererParams()
params.gi_samples = 1
params.gi_weight = 0.3
params.gi_distance = 500
params.shadows = 1.0
params.soft_shadows = 0.3
params.fog_start = 1500
params.fog_thickness = 1500
params.max_bounces = 1
status = be.core_api().set_renderer_params(params)

### Host cell

In [ ]:
# ACE2 receptor definition
ace2_receptor = Protein(
    sources=[pdb_folder + '6m18.pdb'],
    occurences=1,
    position=Vector3(0.0, 6.0, 0.0))

# Membrane definition
membrane_size = scene_size
membrane_height = 80
membrane = Membrane(
    sources=[pdb_folder + 'membrane/popc.pdb'],
    occurences=30000)

# Cell definition
name='Cell'
cell = Cell(
    name=name,
    size=Vector2(membrane_size, membrane_height),
    shape=be.ASSEMBLY_SHAPE_SINUSOIDAL,
    membrane=membrane, receptor=ace2_receptor,
)

# Add cell to scene
status = be.add_cell(
    cell=cell, representation=protein_representation,
    atom_radius_multiplier=protein_radius_multiplier,
    position=Vector3(0.0, -80.0, 0.0),
    random_seed=1
)

status = be.set_protein_color_scheme(
    name, name + '_' + be.NAME_RECEPTOR, 
    be.COLOR_SCHEME_CHAINS, 'OrRd_r', 7)

In [ ]:
glycan_folder = pdb_folder + 'glycans/'
complex_paths = [glycan_folder + 'complex/5.pdb', glycan_folder + 'complex/15.pdb',
                 glycan_folder + 'complex/25.pdb',glycan_folder + 'complex/35.pdb']
high_mannose_paths = [glycan_folder + 'high-mannose/1.pdb', 
                      glycan_folder + 'high-mannose/2.pdb',
                      glycan_folder + 'high-mannose/3.pdb',
                      glycan_folder + 'high-mannose/4.pdb']
hybrid_paths = [glycan_folder + 'hybrid/20.pdb']
o_glycan_paths = [glycan_folder + 'o-glycan/1.pdb']


be.add_multiple_glycans(
    representation=protein_representation, assembly_name=name, 
    glycan_type=be.NAME_GLYCAN_COMPLEX,
    protein_name=be.NAME_RECEPTOR, paths=complex_paths, 
    indices=[53, 90, 103, 322, 432, 690])

be.add_multiple_glycans(
    representation=protein_representation, assembly_name=name,
    glycan_type=be.NAME_GLYCAN_HYBRID,
    protein_name=be.NAME_RECEPTOR, paths=hybrid_paths, 
    indices=[546])

indices = [[164, Quaternion(0.707, 0.0, 0.707, 0.0)],
           [739, Quaternion(0.707, 0.0, 0.707, 0.0)]]
for index in indices:
    o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index[0])
    o_glycan = Sugars(
        assembly_name=name, name=o_glycan_name, 
        source=o_glycan_paths[0],
        protein_name=name + '_' + be.NAME_RECEPTOR, 
        representation=protein_representation,
        chain_ids=[2, 4], site_indices=[index[0]], 
        orientation=index[1])
    be.add_sugars(o_glycan)

## Materials

In [ ]:
be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_BASIC)

## Movie

In [ ]:
key_frames = [
    {
        'apertureRadius': 0.0,
        'direction': [-1.0, 0.0, 0.0],
        'focusDistance': 0.0,
        'origin': [461.35815561123326, 33.59808446214642, 0.9491999386753934],
        'up': [0.0, 1.0, 0.0]
    },
    {
        'apertureRadius': 0.0,
        'direction': [-1.0, 0.0, 0.0],
        'focusDistance': 0.0,
        'origin': [145.56207765382672, -66.33695188520096, 66.94663230017692],
        'up': [0.0, 1.0, 0.0]
    }
]

In [ ]:
mm.build_camera_path(key_frames, 100, 1)

### Coronavirus

In [ ]:
q_start = pyQuaterion(0.707,0.707,0,0)
q_end = pyQuaterion(1,0,0,0)

In [ ]:
from IPython.display import clear_output

name='Coronavirus'
nb_frames = mm.get_nb_frames()

for frame in range(1, nb_frames):
    clear_output()
    print('Frame %d' % frame)
    mm.set_current_frame(frame)
    orientation = pyQuaterion.slerp(q_start, q_end, frame / nb_frames)
    be.add_coronavirus(
        name=name, resource_folder=resource_folder, 
        representation=protein_representation, 
        atom_radius_multiplier=protein_radius_multiplier,
        add_glycans=add_glycans,
        position=Vector3(75.0, 100.0 - frame, 10.0 + frame / 2.0),
        orientation=Quaternion(orientation[0],orientation[1],orientation[2],orientation[3])
    )
    mm.create_snapshot(
        size=[3840, 2160], samples_per_pixel=64, 
        path='/tmp/%05d.png' % frame)